<a href="https://www.kaggle.com/code/aski1140/tmdb-data-science-works?scriptVersionId=132533706" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from scipy import stats
import json
import warnings

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

warnings.filterwarnings("ignore")

In [ ]:
dir_train = "/kaggle/input/tmdb-box-office-prediction/train.csv"
dir_test = "/kaggle/input/tmdb-box-office-prediction/test.csv"

In [ ]:
df_train = pd.read_csv(dir_train)
df_test = pd.read_csv(dir_test)

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train["data"] = "train"
df_test["data"] = "test"

In [ ]:
df_test["revenue"] = np.nan

In [ ]:
df_train.shape, df_test.shape

In [ ]:
df = pd.concat([df_train, df_test], axis = 0)

In [ ]:
df.shape

In [ ]:
df.columns = df.columns.str.lower()

In [ ]:
starting_cols = df_train.columns

# Utils

In [ ]:
def get_dictionary(string):
    try:
        result = eval(string)
    except:
        result = {}
    return result

# EDA 

In [ ]:
df_train.info()

There are 3 feature which is int type, 2 feature which is float and rest of them type is object.

In [ ]:
df_test.info()

## revenue feature

In [ ]:
sns.kdeplot(df[df["data"] == "train"].revenue)
plt.show()

In [ ]:
df["log_revenue"] = np.log1p(df["revenue"])

In [ ]:
sns.kdeplot(df[df["data"] == "train"].log_revenue)
plt.show()

log transformation of revenue feature works to converged to normal distribution.

## belongs_to_collection feature

Some movies has collection but some of them not. We are gonna create feature as a indicator which has movie collection or not

In [ ]:
df["is_have_collection"] = df["belongs_to_collection"].apply(lambda x: 1 if get_dictionary(x) != {} else 0)

In [ ]:
sns.barplot(data = df[df["data"] == "train"], x = "is_have_collection", y = "revenue")
plt.title("Revenue by Is_have_collection")
plt.show()

As we see, it is important that if movies has collection

In [ ]:
df = df.drop("belongs_to_collection", axis = 1)

## homepage feature

Some movies has homepage but some of them not. We are gonan create feature using this knowledge

In [ ]:
df["homepage"].head()

In [ ]:
df["is_have_homepage"] = df["homepage"].apply(lambda x: 1 if "http" in str(x) else 0)

In [ ]:
sns.barplot(data = df[df["data"] == "train"], x = "is_have_homepage", y = "revenue")
plt.title("Revenue by Is_have_homepage")
plt.show()

It seems important for a movie to have a homepage.

In [ ]:
df = df.drop("homepage", axis = 1)

## genres feature

In [ ]:
df[df["data"] == "train"]["genres"].head()

We should extract genres_name as a movie category. Firstly we should delete NaN values to this feature

In [ ]:
df["genres"] =  df["genres"].map(lambda x: sorted([d["name"] for d in get_dictionary(x)])).map(lambda x: ",".join(map(str,x)))
genres = df["genres"].str.get_dummies(sep  = ",")
df = pd.concat([df, genres], axis = 1, sort = False)

In [ ]:
df.shape

Columns number increased, so it did work succesfully

In [ ]:
df = df.drop("genres", axis = 1)

## imdb_id feature

In [ ]:
df[df["data"] == "train"]["imdb_id"].head()

This feature seems like unnecessary.

In [ ]:
df = df.drop("imdb_id", axis = 1)

## original_language

In [ ]:
plt.figure(figsize = (15,8))
df[df["data"] == "train"]["original_language"].value_counts().plot(kind="bar")
plt.title("Count of Original Language of Movies")
plt.show()

Most of movies language is English as expected. We are gonna convert that type of feature.  

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
df["original_language"] = label_encoder.fit_transform(df["original_language"])

In [ ]:
sns.scatterplot(x = "original_language", y = "revenue", data = df[df["data"] == "train"])
plt.xlabel("Original_Language")
plt.ylabel("Revenue")
plt.show()

## poster_path feature

In [ ]:
df[df["data"] == "train"]["poster_path"].head()

This feature seems like irrelevant. So we are gonna add this feature to drop list

In [ ]:
df = df.drop("poster_path", axis = 1)

### original_title feature

In [ ]:
df[df["data"] == "train"]["original_title"].head()

In [ ]:
len(df[df["data"] == "train"]["original_title"].unique()), df[df["data"] == "train"]["original_title"].shape[0]

There are 25 movies which has same title. This can be movies which produced second or third times. This feature seems like unnecessary, so we can add this feature to drop list

In [ ]:
df = df.drop("original_title", axis = 1)

### popularity feature

In [ ]:
sns.scatterplot(x = "popularity", y = "revenue", data = df[df["data"] == "train"])
plt.show()

Mostly, popularity is between 0 to 50.

In [ ]:
sns.scatterplot(x = "popularity", y = "log_revenue", data = df[df["data"] == "train"])
plt.show()

## production_companies feature

This feature has 149 NaN values in train dataset, and 258 NaN values in test set.

In [ ]:
df[df["data"] == "train"]["production_companies"].head()

We can extract production companies' name and we can doing one hot encoding process to extract useful information

In [ ]:
list_of_companies = list(df['production_companies']. \
                         apply(lambda x: [i['name'] for i in get_dictionary(x)] if x != {} \
                               else []).values)

In [ ]:
element_frequencies = Counter([i for j in list_of_companies for i in j]).most_common()

In [ ]:
element_frequencies[:10]

In [ ]:
df["production_companies"] = df["production_companies"]. \
                                    map(lambda x: [i["name"] for i in get_dictionary(x)]). \
                                    map(lambda x: ",".join(map(str,x)))

In [ ]:
df["mean_production_company_frequencies"] = df["production_companies"]. \
                apply(lambda x: np.mean([count for element, count in element_frequencies if element in x]) \
                      if x != "" else 0)

In [ ]:
df["mean_production_company_frequencies"].head()

In [ ]:
sns.scatterplot(x = "mean_production_company_frequencies", y = "log_revenue", data = df[df["data"] == "train"])
plt.title("Revenue by mean_production_company_frequencies")
plt.show()

In [ ]:
df.shape

Columns feature has increased, so it did work succesfully.

In [ ]:
df = df.drop("production_companies", axis = 1)

## production_countries feature

In [ ]:
df[df["data"] == "train"]["production_countries"].head()

We should extract production country name and we can use this information

In [ ]:
list_of_countries = list(df['production_countries']. \
                         apply(lambda x: [i['name'] for i in get_dictionary(x)] if x != {} \
                               else []).values)

In [ ]:
element_frequencies_country = Counter([i for j in list_of_countries for i in j]).most_common()

In [ ]:
element_frequencies_country[:10]

In [ ]:
df["production_countries"] = df["production_countries"]. \
                                    map(lambda x: [i["name"] for i in get_dictionary(x)]). \
                                    map(lambda x: ",".join(map(str,x)))

In [ ]:
df["mean_production_country_frequencies"] = df["production_countries"]. \
                apply(lambda x: np.mean([count for element, count in element_frequencies_country if element in x]) \
                      if x != "" else 0)

In [ ]:
df["mean_production_country_frequencies"].head()

In [ ]:
sns.scatterplot(x = "mean_production_country_frequencies", y = "log_revenue", data = df[df["data"] == "train"])
plt.title("Log_Revenue by mean_production_country_frequencies")
plt.show()

In [ ]:
#df["production_countries"] = df["production_countries"].map(lambda x: [i["name"] for i in get_dictionary(x)]) \
#                                                        .map(lambda x: ",".join(map(str, x)))
#production_countries = df["production_countries"].str.get_dummies(sep  =",")
#df = pd.concat([df, production_countries], axis = 1)
                                                                

In [ ]:
df.shape

In [ ]:
df = df.drop("production_countries", axis = 1)

## release_date feature

In [ ]:
df["release_date"].head()

We can extract useful information with using this feature. For example, year, month, day, day of week.

In [ ]:
def fix_dates(dates):
    try:
        year = str(dates).split("/")[2]
        if int(year) <= 19:
            date = dates[:-2] + "20" + year
        else:
            date = dates[:-2] + "19" + year
    except:
        date = np.nan
    return date

In [ ]:
df["release_date"] = df["release_date"].apply(fix_dates)

In [ ]:
df["release_date"].head()

In [ ]:
# Convert data type to datetime
df["release_date"] = pd.to_datetime(df["release_date"])

In [ ]:
df["release_date"].info()

There is one row with NaN values on release_date feature. I am gonna delete this row

In [ ]:
def datetime_features(dataframe):
    dataframe["month"] = dataframe["release_date"].dt.month
    dataframe["year"] = dataframe["release_date"].dt.year
    dataframe["day"] = dataframe["release_date"].dt.day
    dataframe["dayofweek"] = dataframe["release_date"].dt.dayofweek
    dataframe["quarter"] = dataframe["release_date"].dt.quarter
    dataframe["weekofyear"] = dataframe["release_date"].dt.weekofyear
    return dataframe

In [ ]:
df = datetime_features(df)

In [ ]:
def draw_boxplot(dataframe, col):
    plt.figure(figsize = (15,6))
    sns.boxplot(data = dataframe, x = col, y = "log_revenue")
    plt.xlabel(col)
    plt.ylabel("log_revenue")
    plt.title(f"Log of Revenue by {col}")
    plt.show()

### month feature

In [ ]:
draw_boxplot(df[df["data"] == "train"], "month")

Movies which produced in June, July and December brings more revenue. Movies which produced in January and September brings less revenue.

In [ ]:
plt.figure(figsize = (20,12))
sns.countplot(x = df[df["data"] == "train"]["month"].sort_values())
plt.title("Number of Movies Throught The Month")
loc, labels = plt.xticks()
plt.xticks(fontsize=12,rotation=0)
plt.show()

In [ ]:
df[df["data"] == "train"][["month", "revenue"]].groupby(by = "month").mean().plot()
plt.title("Mean Revenue by Months")
plt.show()

Seems like a there is a seasonal effect when we calculate mean revenue by months

In [ ]:
# Seasonal features by month
df['month_sin'] = np.sin(2*np.pi*df.month/12)
df['month_cos'] = np.cos(2*np.pi*df.month/12)

In [ ]:
draw_boxplot(df[df["data"] == "train"], "month_sin")

In [ ]:
draw_boxplot(df[df["data"] == "train"], "month_cos")

### year feature

In [ ]:
df.loc[df["data"] == "train", ["year", "log_revenue"]].groupby(by = "year", group_keys = True).sum().plot()
plt.title("Total Revenue of Movies by Years")
plt.show()

As we get closer to the present, the total revenue from movies increases throughout the year. But revenue have decreased on last year

In [ ]:
plt.figure(figsize = (20,12))
sns.countplot(x = df[df["data"] == "train"]["year"].sort_values())
plt.title("Number of Movies Throught The Year")
loc, labels = plt.xticks()
plt.xticks(fontsize=12,rotation=90)
plt.show()

As we get closer to the present, the total count of movies increases throughout the year.

### dayofweek feature

In [ ]:
draw_boxplot(df[df["data"] == "train"], "dayofweek")

Movies which released on Tuesday, Wednesday and Thursday have higher revenue than other movies.

In [ ]:
df["is_weekday_TWT"] = df["dayofweek"].apply(lambda x: 1 if x in [2, 3, 4] else 0)

In [ ]:
grid = sns.FacetGrid(data = df, col = "is_weekday_TWT")
grid.map(sns.histplot, "log_revenue")
plt.show()

As we see, is_weekday_TWT feature has really impact on log_revenue

In [ ]:
plt.figure(figsize = (15,6))
sns.countplot(x = df[df["data"] == "train"].dayofweek)
loc, labels = plt.xticks()
loc, labels = loc, ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
plt.xticks(loc, labels, fontsize = 10)
plt.title("Total Number of Movies by DayofWeek")
plt.show()

Most of movies released on Friday. Count of movies which released on Saturday, Sunday and Monday is relatively few.

### quarter feature

In [ ]:
draw_boxplot(df[df["data"] == "train"], "quarter")

In this graph, first quarter cover January, February and March and last quarter cover October, November and December.There was an increase in revenues in the second quarter compared to the first quarter. But There was a decrease in revenues in the third quarter compared to the second quarter and there was an increase in revenues in the fourth quarter compared to the third quarter. Seems like there is a seasonal effect on revenue.

In [ ]:
df[df["data"] == "train"][["quarter", "revenue"]].groupby(by = "quarter").mean().plot()
plt.title("Mean Revenue by Quarters")
plt.show()

In [ ]:
# Seasonal features by month
df['quarter_sin'] = np.sin(2*np.pi*df.quarter/4)
df['quarter_cos'] = np.cos(2*np.pi*df.quarter/4)

In [ ]:
draw_boxplot(df[df["data"] == "train"], "quarter_sin")

In [ ]:
draw_boxplot(df[df["data"] == "train"], "quarter_cos")

In [ ]:
plt.figure(figsize = (15,6))
sns.countplot(x = df[df["data"] == "train"].quarter)
plt.title("Total Number of Movies by Quarters")
plt.show()

Count of movies which released on 3 and 4 quarter are relatively much higher than other two quarters

### day feature

In [ ]:
draw_boxplot(df[df["data"] == "train"], "day")

In [ ]:
plt.figure(figsize = (15,6))
sns.countplot(x = df[df["data"] == "train"].day)
plt.title("Total Number of Movies by Days")
plt.show()

Count of movies which released on first day of months are higher than others. Producers must be choosing the first day of the month to release movie.

### weekofyear feature

In [ ]:
draw_boxplot(df[df["data"] == "train"], "weekofyear")

In [ ]:
df = df.drop(["release_date", "dayofweek"], axis = 1)

## runtime feature

In [ ]:
sns.scatterplot(x = "runtime", y = "revenue", data = df[df["data"] == "train"])
plt.title("Revenue by Runtime")
plt.show()

Mostly, Runtime is between 50 to 150. Some of them are 0. We should look at this films

In [ ]:
df.loc[df["data"] == "train"].groupby(by = "year")["runtime"].mean().plot(color = "b")
plt.xlabel("Release Year")
plt.ylabel("Revenue")
plt.show()

As we get closer to the present, it seems that a standard has been achieved in the duration of the films.

## spoken_languages

In [ ]:
df["spoken_languages"].head(10)

In [ ]:
df.shape

I want to add number of languages spoken in movies

In [ ]:
df["num_spoken_languages"] = df["spoken_languages"].apply(lambda x: len(get_dictionary(x)) if get_dictionary(x) != {} else 0 )

In [ ]:
sns.catplot(x = "num_spoken_languages", y = "revenue", data = df[df["data"] == "train"])
plt.title("Revenue per num_spoken_languages")
plt.show()

As we see, movies which number of languages spoken is 1 more than others.   

In [ ]:
list_spoken_languages = list(df['spoken_languages']. \
                         apply(lambda x: [i['iso_639_1'] for i in get_dictionary(x)] if x != {} \
                               else []).values)

In [ ]:
element_frequencies_spoken_languages = Counter([language for languages in list_spoken_languages for language in languages]).most_common(15)

In [ ]:
elements = [element for element, frequency in element_frequencies_spoken_languages]

In [ ]:
df["spoken_languages"] = df["spoken_languages"].apply(lambda x: [language["iso_639_1"] for language in get_dictionary(x)] if x != {} else []). \
                                                apply(lambda x: ",".join(map(str,x)))

In [ ]:
for element in elements:
    df[f"spoken_language_{element}"] = df["spoken_languages"].apply(lambda x: 1 if element in x else 0)

In [ ]:
spoken_language_cols = df.columns[-15:]

In [ ]:
fig, axes = plt.subplots(nrows = 5, ncols = 3, figsize = (15, 24))
axes = axes.flatten()
for i, ax in enumerate(axes):
    sns.barplot(data=df[df["data"] == "train"], x=spoken_language_cols[i], y='revenue', ax = ax)
plt.show()

In [ ]:
df = df.drop("spoken_languages", axis = 1)

## status feature

In [ ]:
df[df["data"] == "train"].status.value_counts()

There are 4 movies with rumored status. Now, look at revenue of this movies.

In [ ]:
df.loc[(df["data"] == "train") & (df["status"] == "Rumored"), "revenue"]

Movies with rumored status has a revenue. This is weird.

In [ ]:
df[df["data"] == "test"].status.value_counts()

I am gonna drop movies with rumored status

In [ ]:
df = df.drop("status", axis = 1)

## tagline feature

In [ ]:
df["tagline"].head()

In [ ]:
df["tagline"].info()

Some of movies has tagline, some of them not. I am gonna do feature engineering with this knowledge

In [ ]:
df["has_tagline"] = 1
df.loc[df["tagline"].isnull(), "has_tagline"] = 0

In [ ]:
df.has_tagline.value_counts()

Works which we did is proper.

In [ ]:
df = df.drop("tagline", axis = 1)

## title feature

In [ ]:
df["title"].head()

In [ ]:
df["title"].info()

I am gonna add this feature to drop list. I cant think of anything about this feature 

In [ ]:
df = df.drop("title", axis = 1)

## keywords feature

In [ ]:
df["keywords"].head()

In [ ]:
df["keywords"].info()

We can exrtact number of keywords. And after that we can find band for numbers of keywords.

In [ ]:
df["num_keywords"] = df["keywords"].apply(lambda x: len(get_dictionary(x)) if x != {} else 0)

In [ ]:
df["num_keywords_band"] = pd.qcut(df["num_keywords"], q = 5)

In [ ]:
df[["num_keywords_band", "revenue"]].groupby(by = "num_keywords_band").mean().sort_values(by = "revenue", ascending = False)

In [ ]:
df.loc[df["num_keywords"] <= 2, "keywords"] = 1
df.loc[(df["num_keywords"] > 2) & (df["num_keywords"] <= 5), "keywords"] = 2
df.loc[(df["num_keywords"] > 5) & (df["num_keywords"] <= 7), "keywords"] = 3
df.loc[(df["num_keywords"] > 7) & (df["num_keywords"] <= 11), "keywords"] = 4
df.loc[(df["num_keywords"] > 11) & (df["num_keywords"] <= 149), "keywords"] = 5

In [ ]:
plt.figure(figsize = (15, 6))
sns.barplot(data = df[df["data"] == "train"], x = "keywords", y = "revenue")
plt.title("Revenue per Keywords")
plt.show()

In [ ]:
df = df.drop(["num_keywords_band", "num_keywords"], axis = 1)

## cast feature

In [ ]:
for i, e in enumerate(df['cast'][:1]):
    print(i, e)

In [ ]:
df["num_cast"] = df["cast"].apply(lambda x: len(get_dictionary(x)) if x != {} else 0)

In [ ]:
df["num_cast_band"] = pd.qcut(df["num_cast"], q = 5)

In [ ]:
df[["num_cast_band", "revenue"]].groupby(by = "num_cast_band").mean().sort_values(by = "revenue", ascending = False)

In [ ]:
df["cat_cast"] = 1
df.loc[df["num_cast"] <= 10, "cat_cast"] = 1
df.loc[(df["num_cast"] > 10) & (df["num_cast"] <= 14), "cat_cast"] = 2
df.loc[(df["num_cast"] > 14) & (df["num_cast"] <= 19), "cat_cast"] = 3
df.loc[(df["num_cast"] > 19) & (df["num_cast"] <= 28), "cat_cast"] = 4
df.loc[(df["num_cast"] > 28) & (df["num_cast"] <= 165), "cat_cast"] = 5

In [ ]:
plt.figure(figsize = (15, 6))
sns.barplot(data = df[df["data"] == "train"], x = "cat_cast", y = "revenue")
plt.title("Revenue per Cat_cast")
plt.show()

Movies with less than 10 people in cast have the lowest revenue, but movies with more than 28 people in cast have highest revenue 

In [ ]:
df = df.drop(["num_cast", "num_cast_band"], axis = 1)

Now, we can look at the number of films per actor and we can do feature enginnering with this knowledge

In [ ]:
list_actor = list(df["cast"].apply(lambda x: [cast["name"] for cast in get_dictionary(x)] \
                                  if x != {} else []).values)

In [ ]:
castedperson_frequency = Counter([castedperson for casteam in list_actor for castedperson in casteam]).most_common()

In [ ]:
castedperson_frequency[:15]

In [ ]:
df["castedperson_name"] = df["cast"].apply(lambda x: [castedperson["name"] for castedperson in get_dictionary(x)] if get_dictionary(x) != {} else []). \
                                           apply(lambda x: ",".join(map(str, x)))

In [ ]:
df["castedperson_name"].head()

In [ ]:
df["mean_frequencyof_castedperson"] = df["castedperson_name"]. \
                     apply(lambda x: np.mean([freq for person, freq in castedperson_frequency if person in x])) 
                                        

In [ ]:
df["mean_frequencyof_castedperson"].head()

In [ ]:
sns.scatterplot(data = df[df["data"] == "train"], y = "mean_frequencyof_castedperson", x = "revenue")
plt.title("Revenue by Mean_frequencyof_castedperson")
plt.show()

Now, I want to add binary variables which indicator for most common 15 castedperson

In [ ]:
most_common_15 = Counter([castedperson for casteam in list_actor for castedperson in casteam]).\
                            most_common(15)

In [ ]:
most_common_15

In [ ]:
for common_actor, freq in most_common_15:
    df[f"is_{common_actor}_play"] = df["castedperson_name"].apply(lambda x: 1 if common_actor in x.split(",") else 0)

In [ ]:
df[df.columns[-15:]]

In [ ]:
actor_cols = [col for col in df.columns if "play" in col]
fig, axes = plt.subplots(nrows = 5, ncols = 3, figsize = (15, 24))
axes = axes.flatten()
for i, ax in enumerate(axes):
    sns.barplot(data=df[df["data"] == "train"], x=actor_cols[i], y='revenue', ax = ax)
plt.show()

Except Robert De Niro, the actors have had a positive impact on the revenues of their films.

Now we can add castedperson's gender as a feature. Balance of gender can be important for the movies. There are 3 type gender which 0 is unspecified, 1 is female and 2 ise male. ((https://www.kaggle.com/c/tmdb-box-office-prediction/discussion/80983#475572))

In [ ]:
df["gender_0_cast"] = df["cast"].apply(lambda x: sum([1 for i in get_dictionary(x) if i["gender"] == 0]) if x != {} else 0)
df["gender_1_cast"] = df["cast"].apply(lambda x: sum([1 for i in get_dictionary(x) if i["gender"] == 1]) if x != {} else 0)
df["gender_2_cast"] = df["cast"].apply(lambda x: sum([1 for i in get_dictionary(x) if i["gender"] == 2]) if x != {} else 0)

In [ ]:
gender_cols = [col for col in df.columns if "gender_" in col]
fig, axes = plt.subplots(ncols = 3, figsize = (15, 6))
axes = axes.flatten()
for i, ax in enumerate(axes):
    sns.scatterplot(data = df[df["data"] == "train"], x = gender_cols[i], y = "revenue", ax = ax)
    plt.title(f"Revenue by {gender_cols[i]}")
plt.show()

In [ ]:
df = df.drop(["cast", "castedperson_name"], axis = 1)

## crew feature

In [ ]:
for i, e in enumerate(df['crew'][:1]):
    print(i, e)

In movies, crew members is important as much as cast members. I want to divide crew members according to jobs. We can determine most 5 common persons according to jobs and using this knowledge we can do feature engineering

In [ ]:
df["crew_name_job"] = df["crew"].apply(lambda x: [(crew["name"], crew["job"]) for crew in get_dictionary(x)] if x != {} else [])

In [ ]:
for i, e in enumerate(df['crew_name_job'][:1]):
    print(i, e)

In [ ]:
list_of_crew_jobs = list(df['crew'].apply(lambda x: [i['job'] for i in get_dictionary(x)] if x != {} else []).values)
most_jobs = [i for i, j in Counter([i for j in list_of_crew_jobs for i in j]).most_common(15)]
most_jobs

In [ ]:
dict_jobs =  {}
for job in most_jobs:
    list_job = list(df["crew"].apply(lambda x: [crew["name"] for crew in get_dictionary(x) if crew["job"] == job] if x != {} else []).values)
    dict_jobs[job] = list_job

In [ ]:
for job, people  in dict_jobs.items():
    person_frequency = Counter([person for i in people for person in i]).most_common(5)
    top_people = [person for person, freq in person_frequency]
    for top_person in top_people:
        df[f"is_{job}_{top_person}"] = df["crew"].apply(lambda x: 1 if len([True for i in get_dictionary(x) 
                                     if i["job"] == job and i["name"] == top_person] if x != {}
                                                                   else []) >= 1 else 0)

In [ ]:
fig, axes = plt.subplots(nrows = 15, ncols = 5)
fig.set_size_inches(35, 90)
for i, jobs in enumerate(most_jobs):
    if i != 14:
        cols_job = list(df.columns[-75 + i * 5 : -70 + i * 5])
    else:
        cols_job = list(df.columns[-5:])
    for j, col in enumerate(cols_job):
        sns.barplot(data = df[df["data"] == "train"], x = col, y = "revenue", ax = axes[i,j])
        #axes[i, j].set_title(f"Revenue by {cols_job[j]}")
        axes[i, j].xaxis.label.set_fontsize(20)
plt.show()

In [ ]:
df = df.drop(["crew_name_job", "crew"], axis = 1)

# Modeling

We are gonna predict log_revenue feature. After that we tranmsform log of revenue to original revenue feature.

In [ ]:
nulls = pd.isnull(df[df["data"] == "train"])
nulls = nulls.sum()
nulls[nulls != 0]

In [ ]:
nulls = pd.isnull(df[df["data"] == "test"])
nulls = nulls.sum()
nulls[nulls != 0]

In [ ]:
df = df.fillna(0)

In [ ]:
nulls = pd.isnull(df)
nulls = nulls.sum()
nulls[nulls != 0]

In [ ]:
drop_list = []
drop_list.extend(["id", "revenue", "log_revenue"])
len(drop_list)

In [ ]:
drop_list += list(df.select_dtypes(include=['object']).columns)
drop_list = list(set(drop_list))
drop_list = [col for col in drop_list if not "keywords" in col]

In [ ]:
selected_columns = df.columns[~df.columns.isin(drop_list)]

In [ ]:
df[selected_columns].head()

In [ ]:
cat_features_binary = []
for col in selected_columns:
    if df[col].nunique() == 2:
        cat_features_binary.append(col)
cat_features_date = ["month",
                     "year",
                     "day",
                     "quarter",
                     "weekofyear"]

df[cat_features_date] = df[cat_features_date].astype(int)
cat_features = cat_features_date + cat_features_binary + ["keywords", "cat_cast"]

for col in cat_features:
    df[col] = df[col].astype('category')

In [ ]:
train = df[df["data"] == "train"]
test = df[df["data"] == "test"]

In [ ]:
train.shape, test.shape

In [ ]:
# I got this codes from https://www.kaggle.com/code/kamalchhirang/
# eda-feature-engineering-lgb-xgb-cat#Feature-Engineering-&-Prediction
# Clean Data
train.loc[train['id'] == 16,'revenue'] = 192864          # Skinning
train.loc[train['id'] == 90,'budget'] = 30000000         # Sommersby          
train.loc[train['id'] == 118,'budget'] = 60000000        # Wild Hogs
train.loc[train['id'] == 149,'budget'] = 18000000        # Beethoven
train.loc[train['id'] == 313,'revenue'] = 12000000       # The Cookout 
train.loc[train['id'] == 451,'revenue'] = 12000000       # Chasing Liberty
train.loc[train['id'] == 464,'budget'] = 20000000        # Parenthood
train.loc[train['id'] == 470,'budget'] = 13000000        # The Karate Kid, Part II
train.loc[train['id'] == 513,'budget'] = 930000          # From Prada to Nada
train.loc[train['id'] == 797,'budget'] = 8000000         # Welcome to Dongmakgol
train.loc[train['id'] == 819,'budget'] = 90000000        # Alvin and the Chipmunks: The Road Chip
train.loc[train['id'] == 850,'budget'] = 90000000        # Modern Times
train.loc[train['id'] == 1007,'budget'] = 2              # Zyzzyx Road 
train.loc[train['id'] == 1112,'budget'] = 7500000        # An Officer and a Gentleman
train.loc[train['id'] == 1131,'budget'] = 4300000        # Smokey and the Bandit   
train.loc[train['id'] == 1359,'budget'] = 10000000       # Stir Crazy 
train.loc[train['id'] == 1542,'budget'] = 1              # All at Once
train.loc[train['id'] == 1570,'budget'] = 15800000       # Crocodile Dundee II
train.loc[train['id'] == 1571,'budget'] = 4000000        # Lady and the Tramp
train.loc[train['id'] == 1714,'budget'] = 46000000       # The Recruit
train.loc[train['id'] == 1721,'budget'] = 17500000       # Cocoon
train.loc[train['id'] == 1865,'revenue'] = 25000000      # Scooby-Doo 2: Monsters Unleashed
train.loc[train['id'] == 1885,'budget'] = 12             # In the Cut
train.loc[train['id'] == 2091,'budget'] = 10             # Deadfall
train.loc[train['id'] == 2268,'budget'] = 17500000       # Madea Goes to Jail budget
train.loc[train['id'] == 2491,'budget'] = 6              # Never Talk to Strangers
train.loc[train['id'] == 2602,'budget'] = 31000000       # Mr. Holland's Opus
train.loc[train['id'] == 2612,'budget'] = 15000000       # Field of Dreams
train.loc[train['id'] == 2696,'budget'] = 10000000       # Nurse 3-D
train.loc[train['id'] == 2801,'budget'] = 10000000       # Fracture
train.loc[train['id'] == 335,'budget'] = 2 
train.loc[train['id'] == 348,'budget'] = 12
train.loc[train['id'] == 470,'budget'] = 13000000 
train.loc[train['id'] == 513,'budget'] = 1100000
train.loc[train['id'] == 640,'budget'] = 6 
train.loc[train['id'] == 696,'budget'] = 1
train.loc[train['id'] == 797,'budget'] = 8000000 
train.loc[train['id'] == 850,'budget'] = 1500000
train.loc[train['id'] == 1199,'budget'] = 5 
train.loc[train['id'] == 1282,'budget'] = 9               # Death at a Funeral
train.loc[train['id'] == 1347,'budget'] = 1
train.loc[train['id'] == 1755,'budget'] = 2
train.loc[train['id'] == 1801,'budget'] = 5
train.loc[train['id'] == 1918,'budget'] = 592 
train.loc[train['id'] == 2033,'budget'] = 4
train.loc[train['id'] == 2118,'budget'] = 344 
train.loc[train['id'] == 2252,'budget'] = 130
train.loc[train['id'] == 2256,'budget'] = 1 
train.loc[train['id'] == 2696,'budget'] = 10000000




#Clean Data
test.loc[test['id'] == 6733,'budget'] = 5000000
test.loc[test['id'] == 3889,'budget'] = 15000000
test.loc[test['id'] == 6683,'budget'] = 50000000
test.loc[test['id'] == 5704,'budget'] = 4300000
test.loc[test['id'] == 6109,'budget'] = 281756
test.loc[test['id'] == 7242,'budget'] = 10000000
test.loc[test['id'] == 7021,'budget'] = 17540562       #  Two Is a Family
test.loc[test['id'] == 5591,'budget'] = 4000000        # The Orphanage
test.loc[test['id'] == 4282,'budget'] = 20000000       # Big Top Pee-wee
test.loc[test['id'] == 3033,'budget'] = 250 
test.loc[test['id'] == 3051,'budget'] = 50
test.loc[test['id'] == 3084,'budget'] = 337
test.loc[test['id'] == 3224,'budget'] = 4  
test.loc[test['id'] == 3594,'budget'] = 25  
test.loc[test['id'] == 3619,'budget'] = 500  
test.loc[test['id'] == 3831,'budget'] = 3  
test.loc[test['id'] == 3935,'budget'] = 500  
test.loc[test['id'] == 4049,'budget'] = 995946 
test.loc[test['id'] == 4424,'budget'] = 3  
test.loc[test['id'] == 4460,'budget'] = 8  
test.loc[test['id'] == 4555,'budget'] = 1200000 
test.loc[test['id'] == 4624,'budget'] = 30 
test.loc[test['id'] == 4645,'budget'] = 500 
test.loc[test['id'] == 4709,'budget'] = 450 
test.loc[test['id'] == 4839,'budget'] = 7
test.loc[test['id'] == 3125,'budget'] = 25 
test.loc[test['id'] == 3142,'budget'] = 1
test.loc[test['id'] == 3201,'budget'] = 450
test.loc[test['id'] == 3222,'budget'] = 6
test.loc[test['id'] == 3545,'budget'] = 38
test.loc[test['id'] == 3670,'budget'] = 18
test.loc[test['id'] == 3792,'budget'] = 19
test.loc[test['id'] == 3881,'budget'] = 7
test.loc[test['id'] == 3969,'budget'] = 400
test.loc[test['id'] == 4196,'budget'] = 6
test.loc[test['id'] == 4221,'budget'] = 11
test.loc[test['id'] == 4222,'budget'] = 500
test.loc[test['id'] == 4285,'budget'] = 11
test.loc[test['id'] == 4319,'budget'] = 1
test.loc[test['id'] == 4639,'budget'] = 10
test.loc[test['id'] == 4719,'budget'] = 45
test.loc[test['id'] == 4822,'budget'] = 22
test.loc[test['id'] == 4829,'budget'] = 20
test.loc[test['id'] == 4969,'budget'] = 20
test.loc[test['id'] == 5021,'budget'] = 40 
test.loc[test['id'] == 5035,'budget'] = 1 
test.loc[test['id'] == 5063,'budget'] = 14 
test.loc[test['id'] == 5119,'budget'] = 2 
test.loc[test['id'] == 5214,'budget'] = 30 
test.loc[test['id'] == 5221,'budget'] = 50 
test.loc[test['id'] == 4903,'budget'] = 15
test.loc[test['id'] == 4983,'budget'] = 3
test.loc[test['id'] == 5102,'budget'] = 28
test.loc[test['id'] == 5217,'budget'] = 75
test.loc[test['id'] == 5224,'budget'] = 3 
test.loc[test['id'] == 5469,'budget'] = 20 
test.loc[test['id'] == 5840,'budget'] = 1 
test.loc[test['id'] == 5960,'budget'] = 30
test.loc[test['id'] == 6506,'budget'] = 11 
test.loc[test['id'] == 6553,'budget'] = 280
test.loc[test['id'] == 6561,'budget'] = 7
test.loc[test['id'] == 6582,'budget'] = 218
test.loc[test['id'] == 6638,'budget'] = 5
test.loc[test['id'] == 6749,'budget'] = 8 
test.loc[test['id'] == 6759,'budget'] = 50 
test.loc[test['id'] == 6856,'budget'] = 10
test.loc[test['id'] == 6858,'budget'] =  100
test.loc[test['id'] == 6876,'budget'] =  250
test.loc[test['id'] == 6972,'budget'] = 1
test.loc[test['id'] == 7079,'budget'] = 8000000
test.loc[test['id'] == 7150,'budget'] = 118
test.loc[test['id'] == 6506,'budget'] = 118
test.loc[test['id'] == 7225,'budget'] = 6
test.loc[test['id'] == 7231,'budget'] = 85
test.loc[test['id'] == 5222,'budget'] = 5
test.loc[test['id'] == 5322,'budget'] = 90
test.loc[test['id'] == 5350,'budget'] = 70
test.loc[test['id'] == 5378,'budget'] = 10
test.loc[test['id'] == 5545,'budget'] = 80
test.loc[test['id'] == 5810,'budget'] = 8
test.loc[test['id'] == 5926,'budget'] = 300
test.loc[test['id'] == 5927,'budget'] = 4
test.loc[test['id'] == 5986,'budget'] = 1
test.loc[test['id'] == 6053,'budget'] = 20
test.loc[test['id'] == 6104,'budget'] = 1
test.loc[test['id'] == 6130,'budget'] = 30
test.loc[test['id'] == 6301,'budget'] = 150
test.loc[test['id'] == 6276,'budget'] = 100
test.loc[test['id'] == 6473,'budget'] = 100
test.loc[test['id'] == 6842,'budget'] = 30

In [ ]:
X = train[selected_columns]
y = train.revenue
X_test = test[selected_columns]
y_test = test.revenue

In [ ]:
X.shape, X_test.shape

In [ ]:
params = {'learning_rate': 0.03,
          'objective':'MAE',
          'depth': 6,
          'early_stopping_rounds':1000,
          'iterations': 10000,
          'use_best_model': True,
          'eval_metric': "MAPE",
          'random_state': 986,
          'allow_writing_files': False,
          'thread_count':24
          }

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    """
    Calculate the mean absolute percentage error (MAPE).
    
    Parameters:
    - y_true: Array-like of shape (n_samples,), representing the true target values.
    - y_pred: Array-like of shape (n_samples,), representing the predicted values.
    
    Returns:
    - mape: Mean absolute percentage error.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Calculate absolute percentage error
    ape = np.abs((y_true - y_pred) / y_true)
    
    # Calculate mean of absolute percentage error
    mape = np.mean(ape) * 100
    
    return mape

In [ ]:
def catboost_trainer(X,
                     y,
                     submission_df,
                     cv,
                     model_params,
                     feature_list,
                     cat_features,
                     scorer,
                     target_transform=False):
    """
    Catboost Trainer.
    
    ---------
    :param X: training data
    :param y: target
    :param submission_df: test dataframe to be predicted
    :param cv: scikitlearn cross validation object
    :param model_params: dict of catboost model parameters
    :param feature_list: used list of features for training and inference
    :param cat_features: categorical features
    :param scorer: scikitlearn evaluation metric
    :param target_transform: Target
    :return: CV score list, models and submission predictions 
    """
    
    score_list = []
    fold = 1
    unseen_preds = []
    importance = []
    val_results = []
    train_results = []
    models = []
    
    for train_index, test_index in cv.split(X):
        X_train,X_val = X.iloc[train_index][feature_list],X.iloc[test_index][feature_list]
        y_train,y_val = y.iloc[train_index],y.iloc[test_index]
        sub_df_subset = submission_df[feature_list]
        print(f"Training data shape: {X_train.shape}, Validation data shape: {X_val.shape}")
        
        if target_transform:
            y_train = np.log1p(y_train)
            y_val = np.log1p(y_val)
        
        
        model = CatBoostRegressor(**model_params,
                                cat_features=cat_features
                               )
        model.fit(X_train,y_train,
                eval_set=[(X_val,y_val)],
                verbose=500)
        models.append(model)
        forecast_pred = model.predict(sub_df_subset)
        if target_transform:
            forecast_pred = np.expm1(forecast_pred)
        unseen_preds.append(forecast_pred)
        
        val_result = model.predict(X_val)
        if target_transform:
            val_result = np.expm1(val_result)
        
        train_result = model.predict(X_train)
        if target_transform:
            train_result = np.expm1(train_result)
            
        train_results.append(train_result)
        if target_transform:    
            y_train = np.expm1(y_train)
            y_val = np.expm1(y_val)
            
        score = np.sqrt(scorer(y_val,val_result))
        score_t = np.sqrt(scorer(y_train,train_result))
        
        print(f"Score Valid FOLD-{fold}:{score}")
        print(f"Score Train FOLD-{fold}:{score_t}")
        score_list.append(score)
        importance.append(model.get_feature_importance())
        fold += 1
        print('*'*50)
    print("Mean MAPE:", np.mean(score_list),"Std MAPE:",np.std(score_list))
    return score_list, models, unseen_preds

In [ ]:
score_list, models, unseen_preds = catboost_trainer(X=X,
                                                  y=y,
                                                  submission_df= df[df["data"] == "test"],
                                                  cv=KFold(n_splits=5, shuffle=True, random_state=42),
                                                  model_params=params,
                                                  feature_list=selected_columns,
                                                  cat_features=cat_features,
                                                  scorer=mean_absolute_percentage_error,
                                                  target_transform=True)

In [ ]:
importance = [model.get_feature_importance() for model in models]

f_importance = pd.concat([pd.Series(X[selected_columns].columns.to_list(),name='Feature'),
                          pd.Series(np.mean(importance,axis=0),name="Importance")],
                         axis=1).sort_values(by='Importance',
                                             ascending=True)

sns.barplot(f_importance.tail(20),x='Importance',y='Feature')
plt.title("First 20 Important Features - CatBoost Average of Folds")

plt.show()

In [ ]:
sub = pd.read_csv('../input/tmdb-box-office-prediction/sample_submission.csv')
sub["revenue"] = np.mean(unseen_preds, axis = 0)

In [ ]:
sub.head()

In [ ]:
sub.to_csv("submission.csv", index = False)